In [1]:
import json
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf

import tensorflow_hub as hub
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

from official.modeling import tf_utils
from official import nlp
from official.nlp import bert

# Load the required submodules
import official.nlp.optimization
import official.nlp.bert.bert_models
import official.nlp.bert.configs
import official.nlp.bert.run_classifier
import official.nlp.bert.tokenization
import official.nlp.data.classifier_data_lib
import official.nlp.modeling.losses
import official.nlp.modeling.models
import official.nlp.modeling.networks

/home/ning/envs/anaconda3/lib/python3.7/site-packages/tensorflow_addons/utils/ensure_tf_install.py:44: UserWarning: You are currently using a nightly version of TensorFlow (2.4.0-dev20200705). 
TensorFlow Addons offers no support for the nightly versions of TensorFlow. Some things might work, some other might not. 
If you encounter a bug, do not file an issue on GitHub.
  UserWarning,


In [2]:
friend_data_path = r'/home/ning/dataset'
gs_folder_bert = r"/home/ning/BERT/uncased_L-12_H-768_A-12"
bert_path = r"/home/ning/BERT/uncased_L-12_H-768_A-12"

In [3]:
train_df = pd.read_pickle(os.path.join(friend_data_path,'friends_train.pkl'))
test_df = pd.read_pickle(os.path.join(friend_data_path,'friends_test.pkl'))


tokenizer = bert.tokenization.FullTokenizer(
    vocab_file=os.path.join(gs_folder_bert, "vocab_friends.txt"),
     do_lower_case=True)
print("Vocab size:", len(tokenizer.vocab))

Vocab size: 30522


In [34]:

def encode_sentence_with_speaker(speaker,utterance,tokenizer,sos):
    if sos:
        tokens = ['[CLS]']
    else:
        tokens = []

    if speaker not in ['other','None']:
        spk_token = '['+speaker+']'
        tokens.append(spk_token)
        tokens.append('[says]')
        tokens.extend(list(tokenizer.tokenize(utterance)))
        tokens.append('[SEP]')
    
    else:
        tokens.extend(list(tokenizer.tokenize(utterance)))
    
    return tokenizer.convert_tokens_to_ids(tokens)
    
    


def bert_encode(dataframe, tokenizer):
    num_examples = len(dataframe.index)
    sentence1 = tf.ragged.constant([encode_sentence_with_speaker(s[0],s[1],tokenizer,True) for s in dataframe.values])

    sentence2 = tf.ragged.constant([encode_sentence_with_speaker(s[2],s[3],tokenizer,False) for s in dataframe.values])
    
    input_word_ids = tf.concat([sentence1, sentence2], axis=-1)
    
    input_mask = tf.ones_like(input_word_ids).to_tensor()
    
    type_s1 = tf.zeros_like(sentence1)
    type_s2 = tf.ones_like(sentence2)
    input_type_ids = tf.concat([type_s1, type_s2], axis=-1).to_tensor()
    
    
    inputs = {
        'input_word_ids': input_word_ids.to_tensor(),
        'input_mask': input_mask,
        'input_type_ids': input_type_ids}

    return inputs


In [35]:
train_target = train_df['emotion']
test_target = test_df['emotion']
train_features = bert_encode(train_df, tokenizer)
test_features = bert_encode(train_df, tokenizer)

ems = train_target.unique()
def convert(emotion):
    return np.where(ems == emotion)[0][0]

train_labels = np.array(list(map(convert,train_target)))
test_labels = np.array(list(map(convert,test_target)))

In [36]:
bert_config_file = os.path.join(bert_path, "bert_config.json")
config_dict = json.loads(tf.io.gfile.GFile(bert_config_file).read())

bert_config = bert.configs.BertConfig.from_dict(config_dict)

config_dict

{'attention_probs_dropout_prob': 0.1,
 'hidden_act': 'gelu',
 'hidden_dropout_prob': 0.1,
 'hidden_size': 768,
 'initializer_range': 0.02,
 'intermediate_size': 3072,
 'max_position_embeddings': 512,
 'num_attention_heads': 12,
 'num_hidden_layers': 12,
 'type_vocab_size': 2,
 'vocab_size': 30522}

In [39]:
num_labels=8

bert_classifier, bert_encoder = bert.bert_models.classifier_model(
    bert_config, num_labels=8)

weights = tf.constant([2065,1238,4938,1230,379,576,244,190],dtype = tf.int32)

ems

array(['non-neutral', 'surprise', 'neutral', 'joy', 'sadness', 'anger',
       'disgust', 'fear'], dtype=object)

In [56]:
# Set up epochs and steps
epochs = 3
batch_size = 32
eval_batch_size = 32

train_data_size = len(train_labels)
steps_per_epoch = int(train_data_size / batch_size)
num_train_steps = steps_per_epoch * epochs
warmup_steps = int(epochs * train_data_size * 0.1 / batch_size)

# creates an optimizer with learning rate schedule
optimizer = nlp.optimization.create_optimizer(
    2.5e-6, num_train_steps=num_train_steps, num_warmup_steps=warmup_steps)

metrics = [tf.keras.metrics.SparseCategoricalAccuracy('accuracy', dtype=tf.float32)]
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
def NLL(y_true, y_pred):
    
    print(y_true)
    node = -tf.nn.log_softmax(y_pred,axis=1)
    
    arr = tf.zeros((batch_size,num_labels),dtype = tf.float32)
    weights = tf.constant([[ 5.2590799 ,  8.77221325,  2.19927096,  8.82926829, 28.65435356, 18.85416667, 44.50819672, 57.15789474]])
    arr[[range(batch_size)],[list(y_true)]] = 1
    tensor = tf.constant(arr*weights)
    
    s = tensor * y_pred
    tf.reduce_sum(s,axis=1)
    return s

bert_classifier.compile(
    optimizer='adam',
    loss=NLL,
    metrics=metrics)

In [57]:
history = bert_classifier.fit(
              train_features, train_labels,
              validation_data=(test_features, test_labels),
              batch_size=32,
              epochs=1)

Tensor("ExpandDims:0", shape=(None, 1), dtype=int64)


OperatorNotAllowedInGraphError: in user code:

    /home/ning/envs/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    <ipython-input-56-6ae05295975a>:24 NLL  *
        arr[[range(batch_size)],[list(y_true)]] = 1
    /home/ning/envs/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/ops.py:504 __iter__  **
        self._disallow_iteration()
    /home/ning/envs/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/ops.py:497 _disallow_iteration
        self._disallow_when_autograph_enabled("iterating over `tf.Tensor`")
    /home/ning/envs/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/ops.py:475 _disallow_when_autograph_enabled
        " indicate you are trying to use an unsupported feature.".format(task))

    OperatorNotAllowedInGraphError: iterating over `tf.Tensor` is not allowed: AutoGraph did convert this function. This might indicate yo

In [8]:
weights/weights.sum()

array([[0.03018393, 0.05034718, 0.01262248, 0.05067464, 0.1644586 ,
        0.10821148, 0.25545005, 0.32805164]])

In [59]:
data = tf.data.Dataset.from_generator([train_features,train_labels],output_types=None)

TypeError: `generator` must be callable.